# ZhangEN

## Index
1. [Define required clock parameters](#Define-required-clock-parameters)
2. [Download necessary data](#Download-necessary-data)
3. [Load data](#-Load-data)
4. [Extract features and weights](#Extract-features-and-weights)
5. [Load weights into pyaging model](#Load-weights-into-pyaging-model)
6. [Add reference values](#Add-reference-values)
7. [Add preprocessing and postprocesssing steps](#Add-preprocessing-and-postprocesssing-steps)
8. [Check all data objects](#Check-all-data-objects)
9. [Write clock dictionary](#Write-clock-dictionary)
10. [Clear directory](#Clear-directory)

Let's first import some packages:

In [1]:
import os
import marshal
import shutil
import json
import torch
import pandas as pd
import pyaging as pya

## Define required clock parameters

Let's define some required information first:

In [2]:
clock_name = 'zhangen'
data_type = 'methylation'
model_class = 'LinearModel'
species = 'Homo sapiens'
year = 2017
approved_by_author = '⌛'
citation = "Zhang, Qian, et al. \"Improved precision of epigenetic clock estimates across tissues and its implication for biological ageing.\" Genome medicine 11 (2019): 1-11."
doi = "https://doi.org/10.1186/s13073-019-0667-1"
notes = None

## Download necessary data

#### Download GitHub repository

In [3]:
github_url = "https://github.com/qzhang314/DNAm-based-age-predictor.git"
github_folder_name = github_url.split('/')[-1].split('.')[0]
os.system(f"git clone {github_url}")

0

#### Download from R package

Let's first write an R script:

In [4]:
%%writefile download.r

data = readRDS("DNAm-based-age-predictor/data.rds")

write.csv(data, "example_data.csv")

Writing download.r


Let's run the R script. For this, R needs to be installed.

In [5]:
os.system("Rscript download.r")

0

## Load data

#### From CSV file

In [6]:
df = pd.read_table('DNAm-based-age-predictor/en.coef', sep=' ')

reference_feature_values_df = pd.read_csv('example_data.csv', index_col=0)

## Extract features and weights

First, let's extract the features and weights:

In [7]:
df['feature'] = df['probe']
df['coefficient'] = df['coef']

df.head()

,probe,coef,feature,coefficient
0,Intercept,65.792950,Intercept,65.792950
1,cg24611351,-0.001811,cg24611351,-0.001811
2,cg24173182,-0.203955,cg24173182,-0.203955
3,cg09604333,-0.703968,cg09604333,-0.703968
4,cg13617776,-0.011524,cg13617776,-0.011524


Then, let's create lists for features and weights. Be careful about the intercept, as it usually shows up as a feature name.

In [8]:
features = df['feature'][1:].tolist()
weights = torch.tensor(df['coefficient'][1:].tolist()).unsqueeze(0)
intercept = torch.tensor([df['coefficient'][0]])

## Load weights into pyaging model

#### Linear model

In [9]:
model = pya.models.LinearModel(input_dim=len(features))

model.linear.weight.data = weights.float()
model.linear.bias.data = intercept.float()

model

LinearModel(
  (linear): Linear(in_features=514, out_features=1, bias=True)
)

## Add reference values

Some clocks have reference values in the case of missing features. It is also possible that these values are for preprocessing features rather than the clock features. Let's add a dictionary with the feature names as the keys.

In [10]:
reference_feature_values_df = reference_feature_values_df.loc[:, features]

reference_feature_values = dict(zip(reference_feature_values_df.columns.tolist(), reference_feature_values_df.mean().tolist()))

## Add preprocessing and postprocesssing steps

The preprocessing and postprocessing objects are dictionaries with the following format, with all items required. It takes in x in the form of a numpy array.

In [11]:
def preprocessing_function(x):
    """
    Scales the input data per row with mean 0 and std 1.
    """
    row_means = np.mean(x, axis=1, keepdims=True)
    row_stds = np.std(x, axis=1, keepdims=True)

    # Avoid division by zero in case of a row with constant value
    row_stds[row_stds == 0] = 1

    x_scaled = (x - row_means) / row_stds
    return x_scaled

preprocessing_function_string = marshal.dumps(preprocessing_function.__code__)

preprocessing_helper_objects = None

preprocessing = {
    'name': 'scale_row',
    'preprocessing_function': preprocessing_function_string,
    'preprocessing_helper_objects': preprocessing_helper_objects
}

Similarly is the case of postprocessing. Remember that your function must be compatible with torch and is applied to each number individually.

In [12]:
postprocessing = None

## Check all data objects

Let's print all data objects to check if they make sense.

#### features

In [13]:
def my_print_function():
    print(f"There are {len(features)} features.")
    print(features)
pya.utils.print_to_scrollable_output(my_print_function)

#### reference_feature_values

In [14]:
def my_print_function():
    if reference_feature_values:
        print(f"There are {len(reference_feature_values)} reference feature values.")
    print(reference_feature_values)
pya.utils.print_to_scrollable_output(my_print_function)

#### preprocessing

In [15]:
def my_print_function():
    print(preprocessing)
    if preprocessing:
        print(preprocessing_helper_objects)
pya.utils.print_to_scrollable_output(my_print_function)

#### postprocessing

In [16]:
def my_print_function():
    print(postprocessing)
    if postprocessing:
        print(postprocessing)
pya.utils.print_to_scrollable_output(my_print_function)

#### weight_dict

In [17]:
def my_print_function():
    for name, param in model.named_parameters():
        print(f"Layer: {name}")
        print(f"Shape: {param.shape}")
        print(param.data)
pya.utils.print_to_scrollable_output(my_print_function)

## Write clock dictionary

Let's put everything together and save:

In [18]:
clock_dict = {
    # Metadata
    'clock_name': clock_name,
    'data_type': data_type,
    'model_class': model_class,
    'species': species,
    'year': year,
    'approved_by_author': approved_by_author,
    'citation': citation,
    'doi': doi,
    "notes": notes,

    # Data
    'reference_feature_values': reference_feature_values if reference_feature_values else None,
    'preprocessing': preprocessing if preprocessing else None, 
    'features': features,
    'weight_dict': model.state_dict(),
    'postprocessing': postprocessing if postprocessing else None,
}

torch.save(clock_dict, f'../weights/{clock_name}.pt')

## Clear directory

Delete all files that are not clock jupyter notebooks:

In [19]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: download.r
Deleted folder: .ipynb_checkpoints
Deleted folder: DNAm-based-age-predictor
Deleted file: example_data.csv
